In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x) 

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def g(x):  
    x_temp = torch.cos(pi*x)
    g_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return g_x

In [10]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x).cuda()
    uxx = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda() 
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda() 
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda()
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda()
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda()
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda()
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda()
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda()
    xd2[:, 3] = torch.ones(Nb)
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx4[:, 3] = torch.ones(xb1.size()[0])
    uxa1 = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 * (1.0) + model(xa1) - g(xa1))**2).cuda()
    uxa2 = (((model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2 * (-1.0) + model(xa2) - g(xa2))**2).cuda()
    uxb1 = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 * (1.0) + model(xb1) - g(xb1))**2).cuda()
    uxb2 = (((model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2 * (-1.0) + model(xb2) - g(xb2))**2).cuda()
    uxc1 = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 * (1.0) + model(xc1) - g(xc1))**2).cuda()
    uxc2 = (((model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2 * (-1.0) + model(xc2) - g(xc2))**2).cuda()
    uxd1 = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 * (1.0) + model(xd1) - g(xd1))**2).cuda()
    uxd2 = (((model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2 * (-1.0) + model(xd2) - g(xd2))**2).cuda()
    
    part_2 = torch.sum((uxa1 + uxa2 + uxb1 + uxb2 + uxc1 + uxc2 + uxd1 + uxd2)[:,0])/xa1.size()[0]
    lambda1 = 500
    return part_1 + lambda1 * part_2 / 8

In [11]:
Data_size = 500
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [12]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [13]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [14]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [15]:
import torch.optim as optim
import torch.nn as nn
import time

In [16]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [17]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [18]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_4D_Robin_batch_size_500.npy", error_save)
np.save("DGM_relative_error_4D_Robin_batch_size_500.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(7097.3936, device='cuda:0')
current relative error is:  tensor(2.0398, device='cuda:0')
current epoch is:  50
current loss is:  tensor(2460.7265, device='cuda:0')
current relative error is:  tensor(1.0306, device='cuda:0')
current epoch is:  100
current loss is:  tensor(1929.2654, device='cuda:0')
current relative error is:  tensor(0.8996, device='cuda:0')
current epoch is:  150
current loss is:  tensor(1688.0414, device='cuda:0')
current relative error is:  tensor(0.8352, device='cuda:0')
current epoch is:  200
current loss is:  tensor(1292.8789, device='cuda:0')
current relative error is:  tensor(0.7794, device='cuda:0')
current epoch is:  250
current loss is:  tensor(971.6925, device='cuda:0')
current relative error is:  tensor(0.7318, device='cuda:0')
current epoch is:  300
current loss is:  tensor(834.1023, device='cuda:0')
current relative error is:  tensor(0.7313, device='cuda:0')
current epoch is:  350
current loss is:  tensor(730.5

current epoch is:  3050
current loss is:  tensor(6.7067, device='cuda:0')
current relative error is:  tensor(0.2309, device='cuda:0')
current epoch is:  3100
current loss is:  tensor(6.6412, device='cuda:0')
current relative error is:  tensor(0.2440, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(5.7107, device='cuda:0')
current relative error is:  tensor(0.2659, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(5.8837, device='cuda:0')
current relative error is:  tensor(0.2553, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(5.7408, device='cuda:0')
current relative error is:  tensor(0.2220, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(5.5837, device='cuda:0')
current relative error is:  tensor(0.2346, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(5.5311, device='cuda:0')
current relative error is:  tensor(0.2323, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(5.1950, devic

current epoch is:  6150
current loss is:  tensor(1.6895, device='cuda:0')
current relative error is:  tensor(0.1828, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(1.7721, device='cuda:0')
current relative error is:  tensor(0.1869, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(2.0749, device='cuda:0')
current relative error is:  tensor(0.1749, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(1.7817, device='cuda:0')
current relative error is:  tensor(0.1677, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(1.6380, device='cuda:0')
current relative error is:  tensor(0.1796, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(1.5523, device='cuda:0')
current relative error is:  tensor(0.1758, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(1.6525, device='cuda:0')
current relative error is:  tensor(0.1761, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(1.6111, devic

current epoch is:  9250
current loss is:  tensor(0.8749, device='cuda:0')
current relative error is:  tensor(0.1504, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(1.0840, device='cuda:0')
current relative error is:  tensor(0.1663, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(0.8686, device='cuda:0')
current relative error is:  tensor(0.1508, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(0.9456, device='cuda:0')
current relative error is:  tensor(0.1604, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(1.1537, device='cuda:0')
current relative error is:  tensor(0.1648, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(1.4022, device='cuda:0')
current relative error is:  tensor(0.1670, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(1.3069, device='cuda:0')
current relative error is:  tensor(0.1616, device='cuda:0')
current epoch is:  9600
current loss is:  tensor(0.9131, devic

current epoch is:  12300
current loss is:  tensor(0.6653, device='cuda:0')
current relative error is:  tensor(0.1334, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(0.6292, device='cuda:0')
current relative error is:  tensor(0.1259, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(1.4177, device='cuda:0')
current relative error is:  tensor(0.1427, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(0.5324, device='cuda:0')
current relative error is:  tensor(0.1262, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(0.6469, device='cuda:0')
current relative error is:  tensor(0.1271, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(0.6301, device='cuda:0')
current relative error is:  tensor(0.1328, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(0.7531, device='cuda:0')
current relative error is:  tensor(0.1198, device='cuda:0')
current epoch is:  12650
current loss is:  tensor(0.621

current epoch is:  15350
current loss is:  tensor(0.4353, device='cuda:0')
current relative error is:  tensor(0.1000, device='cuda:0')
current epoch is:  15400
current loss is:  tensor(0.4950, device='cuda:0')
current relative error is:  tensor(0.1044, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(0.6023, device='cuda:0')
current relative error is:  tensor(0.0894, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(0.5598, device='cuda:0')
current relative error is:  tensor(0.0940, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(0.6026, device='cuda:0')
current relative error is:  tensor(0.1026, device='cuda:0')
current epoch is:  15600
current loss is:  tensor(0.4606, device='cuda:0')
current relative error is:  tensor(0.1005, device='cuda:0')
current epoch is:  15650
current loss is:  tensor(0.4016, device='cuda:0')
current relative error is:  tensor(0.0964, device='cuda:0')
current epoch is:  15700
current loss is:  tensor(0.443

current epoch is:  18400
current loss is:  tensor(0.3738, device='cuda:0')
current relative error is:  tensor(0.0902, device='cuda:0')
current epoch is:  18450
current loss is:  tensor(0.3667, device='cuda:0')
current relative error is:  tensor(0.0925, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(0.3571, device='cuda:0')
current relative error is:  tensor(0.0859, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(0.3545, device='cuda:0')
current relative error is:  tensor(0.0809, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(0.5722, device='cuda:0')
current relative error is:  tensor(0.0741, device='cuda:0')
current epoch is:  18650
current loss is:  tensor(0.4131, device='cuda:0')
current relative error is:  tensor(0.0759, device='cuda:0')
current epoch is:  18700
current loss is:  tensor(0.3433, device='cuda:0')
current relative error is:  tensor(0.0902, device='cuda:0')
current epoch is:  18750
current loss is:  tensor(0.442